In [2]:
import kagglehub
import cv2
import keras
import os
import numpy as np
from sklearn.model_selection import train_test_split
import xml.etree.ElementTree as ET
import tensorflow as tf

c:\Users\Abolfazl\Desktop\projects\Face-Mask-Detection\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Dataset

In [8]:
# دانلود دیتاست
path = kagglehub.dataset_download("andrewmvd/face-mask-detection")

print("Path to dataset files:", path)

100%|██████████| 398M/398M [03:58<00:00, 1.75MB/s] 

Extracting files...


Path to dataset files: C:\Users\Abolfazl\.cache\kagglehub\datasets\andrewmvd\face-mask-detection\versions\1


# Pre-processing

In [ ]:
# جدا کردن تصاویر، اسم تصاویر، لیبل ها و باوندینگ باکس ها توی لیست های جدا

# مسیر دیتاست
images_path = os.path.join(path, 'images') # مسیر تصاویر
annotations_path = os.path.join(path, 'annotations') # مسیر انوتیشن‌ ها

# لیست تصاویر و آماده‌سازی داده‌ ها
faces = []
labels = []

for filename in os.listdir(images_path):
    img_path = os.path.join(images_path, filename) # مسیر تصویر
    img = cv2.imread(img_path) # خوندن تصویر

    # استخراج لیبل و باوندینگ باکس ها از انوتیشنِ تصویر
    xml_file = os.path.join(annotations_path, filename.replace('.png', '.xml'))
    tree = ET.parse(xml_file) # تبدیل انوتیشن به فرمت شی قابل فهم و ساده به صورت درختی
    root = tree.getroot() # ( <annotation>...</annotation> ) ریشه درخت 

    for object in root.findall('object'): # پیدا کردن همه آبجکت های توی ریشه. هر آبجکت یه فرد توی تصویره
        class_name = object.find('name').text # کلاس فرد (با ماسک | بدون ماسک | با ماسک نادرست زده شده) بدون تگ
        bounding_box = object.find('bndbox') # باوندینگ باکس فرد
        
        # مختصات باوندینگ باکس
        x_start = int(bounding_box.find('xmin').text)
        y_start = int(bounding_box.find('ymin').text)
        x_end = int(bounding_box.find('xmax').text)
        y_end = int(bounding_box.find('ymax').text)

        faces.append(cv2.resize(img[y_start:y_end, x_start:x_end], (216, 216))) # اضافه کردن تصویر کراپ شده باوندینگ باکس به لیست چهره ها
    
        # بررسی تعلق به هر کلاس
        if class_name == 'without_mask':
            labels.append([0])
        elif class_name == 'with_mask':
            labels.append([1])
        elif class_name == 'mask_weared_incorrect':
            labels.append([2])


# تبدیل لیست ها به آرایه نامپای برای پردازش توسط مدل
faces = np.array(faces)
labels = np.array(labels)

# نرمالایز کردن تصاویر
faces = faces.astype('float32') / 255.0

# (80%)تقسیم به داده های ترِین(20%) و تست
train_faces, test_faces, train_labels, test_labels = train_test_split(
    faces, labels, test_size=0.2, shuffle=True
)

# برای محاسبه بهتر تابع هزینه توسط مدل One-Hot Endcoding تبدیل لیبل ها به
train_labels_onehot = tf.keras.utils.to_categorical(train_labels, num_classes=3)
test_labels_onehot = tf.keras.utils.to_categorical(test_labels, num_classes=3)